<a href="https://colab.research.google.com/github/mitsuo/juntendo-hds/blob/main/ECG_Baseline_210326_%E6%94%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ydata-profiling

In [ ]:
# ライブラリ読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, pickle, time, gc, copy, sys
import ydata_profiling as ydp
import warnings
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100) # 表示できる表の列数

# データ読み込み
**ファイル**
- `train.csv` - 学習用データ
- `test.csv` - テスト用データ
- `sample_submission.csv` - 提出ファイルのサンプル, 0 から 1 の間の値の予測結果を target 列に代入して提出してください.
- `ecg/` - 心電図データを含むディレクトリ. 各心電図データはnumpy形式で `心電図ID.npy`の名前で保存されています. データの形は(800, 12)です。
 - 第1次元 - 時間解像度: 100 Hz × 8 秒 = 800 timepoint. 単位: mV
 - 第2次元 - 12誘導: I, II III, aVR, aVL, aVF, V1, V2, V3, V4, V5, V6  

**データ内容**
- `Id` - 匿名化された心電図ID. 全ての心電図データはそれぞれ異なる患者から測定されています.
- `target` - 目的変数. 0=正常, 1=心筋梗塞 (test.csv にはありません)
- `age` - 年齢
- `sex` - 性別. 0=男性, 1=女性.
- `label_type` ラベル作成方法. auto=自動解析, human=医師による診断.

# 心電図データのダウンロード
今回は 公全国医療AIコンテスト 2021 (https://www.kaggle.com/competitions/ai-medical-contest-2021/)の心電図データをダウンロードします。このデータは心電図から心筋梗塞かどうかを判定するタスクのためのものです。

まず **linuxコマンド** を使って 公開されているデータをダウンロードしましょう。下のセルでは `ai-medical-contest-2021` というディレクトリをデフォルトのディレクトリ `/content` の下に作成し、そこに関連データをダウンロードしています。

> コマンドとはコンピュータに計算やファイルの操作を行うように指示できるツールです。今回は詳しく触れませんが、colabのようなノートブックではpythonとコマンドを混ぜて使えるのが一つの強みです。

In [ ]:
# ! を先頭につけると一時的に適応される。例えばワーキングディレクトリの移動をしてもその後のコマンドには適応されない。
# ai-medical-contest-2021 ディレクトリを作成します。
!rm -rf /content/ai-medical-contest-2021/
!mkdir /content/ai-medical-contest-2021
!pwd

# % を先頭につけると永続的に適応される。ワーキングディレクトリの移動をしてもその後も適応される。
# ai-medical-contest-2021 ディレクトリに移動します。
%cd /content/ai-medical-contest-2021
!ls

# 心電図データのダウンロード
!wget http://mitsuo.nishizawa.com/juntendo/ai-medical-contest-2021.zip
!unzip ai-medical-contest-2021.zip
!ls

ダウンロードされたファイルは左のバーのフォルダーマークを押すとみることができます。

今回はいろんなPythonモジュールを使って、一つの心音データを図示することを目標にしましょう。

In [ ]:
# trainファイルを読み込む
df_train = pd.read_csv("/content/ai-medical-contest-2021/train.csv")
print("df_train.shape", df_train.shape) # シェイプ = (行数, 列数)を表示する
df_train.head() # 先頭5行を表示する

In [ ]:
# testファイルを読み込む
df_test = pd.read_csv("/content/ai-medical-contest-2021/test.csv")
print("df_test.shape", df_test.shape) # シェイプ = (行数, 列数)を表示する
df_test.head() # 先頭5行を表示する

In [ ]:
# submissionファイルを読み込む
df_sub = pd.read_csv("/content/ai-medical-contest-2021/sample_submission.csv")
print("df_sub.shape", df_sub.shape) # シェイプ = (行数, 列数)を表示する
df_sub.head() # 先頭5行を表示する

In [ ]:
# ECGデータのpathの列を追加.
df_train['path'] = df_train['Id'].apply(lambda x: "/content/ai-medical-contest-2021/ecg/{}.npy".format(x))
df_test['path'] = df_test['Id'].apply(lambda x: "/content/ai-medical-contest-2021/ecg/{}.npy".format(x))
print(df_train['path'][0]) # path列の0行目を表示
df_train.head()

In [ ]:
# trainとtestを連結する
df_traintest = pd.concat([df_train, df_test]).reset_index(drop=True) # reset_index: 行のindexをリセットする
print(df_traintest.shape)
df_traintest.head()

# EDA (Explanatory Data Analysis, 探索的データ解析)

In [ ]:
# ターゲットについて解析
col_target = 'target' # ターゲットの列
col_index = 'Id' # idの列
print("rate of positive: {:.6f}".format(df_train[col_target].mean())) # targetが1である割合

In [ ]:
# 各列の基本情報を表示
# 解析対象はtrain+test
# 列名, 型, nanの数, uniqueな値の数, 実際の値の一部, を表示する
df_tmp = df_traintest  # 解析するDataFrameを指定
for i, col in enumerate(df_tmp.columns): # 各列(column)について
    col_name = col + " " * (22 - len(col)) # カラム名, 見た目上の整形のためにスペースを加える
    type_name = "{}".format(df_tmp[col].dtype) # 型名
    type_name = type_name + " " * (8 - len(type_name)) # 見た目上の整形のためにスペースを加える
    num_unique = len(df_tmp[col].unique()) # ユニークな値の数
    num_nan = pd.isna(df_tmp[col]).sum() # nanの数
    col_head = "{}".format(df_tmp[col].unique()[:5].tolist())[:40] # 実際の値の一部
    print("{:4d}: {} dtype: {} unique: {:8d}, nan: {:6d}, 実際の値: {}".format(
        i, col_name, type_name, len(df_tmp[col].unique()), num_nan, col_head)) # 表示する

In [ ]:
# ydata_profiling でデータの解析を行う
# 解析対象はtrain+test
ydp.ProfileReport(df_traintest)

In [ ]:
# 数値変数についてヒストグラムを表示する
# 解析対象はtrain+test
from scipy.stats import norm
cols_num = ['age'] # 数値変数の列名のリスト, このデータの場合 age のみ
fig = plt.figure(figsize=(20, int(4*int(np.ceil(len(cols_num)/4)))))
for i, col in enumerate(cols_num[:]):
    ax = fig.add_subplot(int(np.ceil(len(cols_num)/4)),4,i+1)
    sns.distplot(
        df_traintest[col], # 表示するデータ
        bins=20, # ヒストグラムのビンの数
        color='black', label='data',
        kde_kws={'label': 'kde','color':'r'},
#         fit=norm,
#         fit_kws={'label': 'norm','color':'red'},
        rug=False
    )

In [ ]:
# カテゴリ変数について棒グラフを表示する
# 解析対象はtrain+test
cols_cat = ['sex', 'label_type'] # 解析する列名
for i, col in enumerate(cols_cat):
    g = sns.catplot(
        x=col,
        kind="count",
        data=df_traintest, # 解析するDataFrame
        height=5,
        palette="muted"
    )
    g.fig.set_figwidth(16)
    g.fig.set_figheight(2)
    plt.show()

In [ ]:
# ターゲットの値ごとに数値変数のバイオリンプロットを表示
# target=0の場合のageの分布、target=1の場合のageの分布を表示する
# 2つの分布が違うという事はその変数がターゲットに強い影響を与えていることを示唆する
# 解析対象はtrainのみ
fig = plt.figure(figsize=(20, int(4*int(np.ceil(len(cols_num)/4)))))
for i, col in enumerate(cols_num):
    ax = fig.add_subplot(int(np.ceil(len(cols_num)/4)),4,i+1)
    sns.violinplot(
        x=col_target,
        y=col,
        data=df_train,
#         scale='count', # 消すと正規化する
    )

In [ ]:
# 数値変数について logistic regression plot
# age を説明変数として target をアウトカムとするロジスティック回帰を行ってそれを表示
# 解析対象はtrainのみ
fig = plt.figure(figsize=(20, int(4*int(np.ceil(len(cols_num)/4)))))
for i, col in enumerate(cols_num[:]):
    ax = fig.add_subplot(int(np.ceil(len(cols_num)/4)),4,i+1)
    sns.regplot(x=col,
                y=col_target,
                data=df_train,
                logistic=True,
                scatter_kws={'s': 10, 'alpha':0.3,'color':'m'},
    )

In [ ]:
# カテゴリ変数について積み上げ棒グラフを表示
# 解析対象はtrainのみ
def stack_bar_plot(df_tmp, col, col_target, ax=None):
    df_tmp[col][pd.isna(df_tmp[col])] = 'nan'
    target_value = df_tmp[col].unique()
    df_agg = df_tmp[df_tmp[col_target].duplicated()==False][[col_target]].sort_values(col_target).reset_index(drop=True)
    for value in target_value:
        col_value = "{}".format(value)
        df_agg_tmp = df_tmp[df_tmp[col]==value].groupby(col_target)[col].agg(len).reset_index()
        df_agg_tmp = df_agg_tmp.sort_values(col_target).reset_index(drop=True)
        df_agg_tmp.columns = [col_target, col_value]
        df_agg = pd.merge(df_agg, df_agg_tmp, on=col_target, how='left')
    df_agg = df_agg.fillna(0)
    col_new = "{}/{}".format(col, col_target)
    df_agg.columns = [col_new] + df_agg.columns[1:].values.tolist()
    df_agg = df_agg.set_index(col_new)
    df_agg.iloc[:] = df_agg.values / df_agg.values.sum(axis=1)[:,np.newaxis]
    ax1 = df_agg.plot.bar(stacked=True, ax=ax)
    ax1.legend(title=col)


cols_cat = ['sex', 'label_type'] # 解析する列名
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
for i, col in enumerate(cols_cat):
    stack_bar_plot(df_train, col, col_target, ax=axes[i])

In [ ]:
# train, testそれぞれのlabel_typeの分布を確認する
g = sns.catplot(
        x='label_type',
        kind="count",
        data=df_train, # 解析するDataFrame
        height=5,
        palette="muted"
)
g.fig.set_figwidth(16)
g.fig.set_figheight(2)
plt.title("train data", fontsize=16)
plt.show()

g = sns.catplot(
        x='label_type',
        kind="count",
        data=df_test, # 解析するDataFrame
        height=5,
        palette="muted"
)
g.fig.set_figwidth(16)
g.fig.set_figheight(2)
plt.title("test data", fontsize=16)
plt.show()

In [ ]:
# 心電図波形を表示する
index = 10 # 表示する行を指定
path_tmp = df_train['path'][index] # 表示する心電図データのpath
ecg_tmp = np.load(path_tmp) # 心電図データのよみこみ
print("i: {}, id: {}".format(index, df_train[col_index][index]))
print("ECG shape: {}".format(ecg_tmp.shape))
print("target: {}".format(df_train[col_target][index]))
lead_list = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'] # 各誘導の名称
plt.figure(figsize=(20,16))
for i in range(12): # 各誘導を可視化
    if i<6:
        plt.subplot(6,2,i*2+1)
    else:
        plt.subplot(6,2,i*2-10)
    plt.plot(ecg_tmp[:,i])
    plt.xticks(np.arange(0, 801, step=100))
    plt.ylabel(lead_list[i]+"  ", rotation=0, fontsize=16)
    plt.minorticks_on()
    plt.ylim(ecg_tmp.min(),ecg_tmp.max())
    plt.grid(which="major", color="black", alpha=0.5)
    plt.grid(which="minor", color="gray", linestyle=":")

# 前処理

In [ ]:
# カテゴリ変数をラベルエンコーディングする (数値に置き換える).
df_traintest['sex'] = df_traintest['sex'].replace('female', 0) # femaleに0を代入
df_traintest['sex'] = df_traintest['sex'].replace('male', 1) # maleに1を代入
df_traintest['sex'] = df_traintest['sex'].astype(int) # 型を整数に変換

df_traintest['label_type'] = df_traintest['label_type'].replace('human', 0) # humanに0を代入
df_traintest['label_type'] = df_traintest['label_type'].replace('auto', 1) # autoに1を代入
df_traintest['label_type'] = df_traintest['label_type'].astype(int) # 型を整数に変換
df_traintest.head()

In [ ]:
# train と test を再度切り分ける
df_train = df_traintest.iloc[:len(df_train)]
df_test = df_traintest.iloc[len(df_train):].reset_index(drop=True)
df_train.head()

In [ ]:
# 全てのECGデータを読み込む
ecg_train = np.zeros([len(df_train), 800, 12], np.float32) # trainの心電図データの代入先. shape=(データ数, 時間方向, 12誘導)
for i in range(len(df_train)): # 全てのtrain dataについて
    path_tmp = df_train['path'][i] # i行目の心電図データのpath
    ecg_tmp = np.load(path_tmp) # i行目の心電図データ
    ecg_train[i] = ecg_tmp # 読み込んだ心電図データをecg_trainのi行目に代入

ecg_test = np.zeros([len(df_test), 800, 12], np.float32) # testの心電図データの代入先. shape=(データ数, 時間方向, 12誘導)
for i in range(len(df_test)): # 全てのtest dataについて
    path_tmp = df_test['path'][i] # i行目の心電図データのpath
    ecg_tmp = np.load(path_tmp) # i行目の心電図データ
    ecg_test[i] = ecg_tmp # 読み込んだ心電図データをecg_trainのi行目に代入
print("ecg_train.shape: {}".format(ecg_train.shape))
print("ecg_test.shape: {}".format(ecg_test.shape))

In [ ]:
# target情報をnumpy形式に変換
target_train = df_train[col_target].values.astype(int) # pandas.Seriesからnp.ndarrayへ変換
print("target_train.shape: {}".format(target_train.shape))

# ベースラインモデルを作成
### クロスバリデーション
![CV](https://jp.mathworks.com/discovery/cross-validation/_jcr_content/mainParsys/image.adapt.480.medium.jpg/1611249616013.jpg)
https://jp.mathworks.com/discovery/cross-validation.html

In [ ]:
# クロスバリデーションを行うためにデータを5分割する
# 4つを学習に用い、1つを検証に要する。これを5回繰り返す。
folds = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=42).split(
    np.arange(len(df_train)),
    y=df_train[col_target]) # 各foldターゲットのラベルの分布がそろうようにする = stratified K fold
)

In [ ]:
# fold 0の学習データと検証データの分割
fold = 0 # fold 0 についての学習を行う

# このfoldにおける学習データと検証データの切り分け
X_train = ecg_train[folds[fold][0]] # 学習データの入力データを抽出
y_train = target_train[folds[fold][0]] # 学習データの正解データを抽出
X_valid = ecg_train[folds[fold][1]] # 検証データの入力データを抽出
y_valid = target_train[folds[fold][1]] # 検証データの正解データを抽出
print("X_train.shape: {}, X_valid.shape: {}".format(X_train.shape, X_valid.shape))
print("y_train.shape: {}, y_valid.shape: {}".format(y_train.shape, y_valid.shape))

In [ ]:
# modelにdataを流すためのdatasetを構築する
BATCH_SIZE = 64 # ミニバッチに含めるデータの数
def augment_fn(X, y):
    """
    augmentation (データ水増し)を設定する
    """
    X_new = tf.image.random_crop(X, (700,12)) # 時間方向に800 timepointからrandomに700 timepointを切り出す
    return (X_new, y)

# train dataset
train_dataset = tf.data.Dataset.from_tensor_slices(( # np
    X_train, # 入力データ
    y_train, # 正解データ
))
train_dataset = train_dataset.shuffle(len(train_dataset), reshuffle_each_iteration=True) # 学習中にデータをシャッフルすることを指定する
train_dataset = train_dataset.map(augment_fn, num_parallel_calls=AUTOTUNE) # augmentationの適用
train_dataset = train_dataset.batch(BATCH_SIZE) # データセットをミニバッチ化してモデルに入力することを指定する

# valid dataset
valid_dataset = tf.data.Dataset.from_tensor_slices((
    X_valid, # 入力データ
    y_valid, # 正解データ
))
valid_dataset = valid_dataset.batch(BATCH_SIZE) # データセットをミニバッチ化してモデルに入力することを指定する (シャッフルはしない)

# test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((
    ecg_test, # 入力データ
    np.zeros(len(ecg_test)), # 正解データ (testに正解データないためダミーデータ)
))
test_dataset = test_dataset.batch(BATCH_SIZE) # データセットをミニバッチ化してモデルに入力することを指定する (シャッフルはしない)


# datasetの読み込みテスト
ecg_batch, target_batch = next(iter(train_dataset)) # 試しにミニバッチを読み込む
print("train ecg_batch.shape: {}".format(ecg_batch.shape))
print("train target_batch.shape: {}".format(target_batch.shape))
ecg_batch, target_batch = next(iter(valid_dataset)) # 試しにミニバッチを読み込む
print("valid ecg_batch.shape: {}".format(ecg_batch.shape))
print("valid target_batch.shape: {}".format(target_batch.shape))

### Convolutional neural networks
![CNN](https://cdn-ak.f.st-hatena.com/images/fotolife/a/acro-engineer/20180318/20180318155516.png)
(LeCun et al. 1998)  

In [ ]:
# deep learning modelの作成
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
def get_model(input_shape=(800, 12)):
    model = tf.keras.models.Sequential([ # レイヤーのリストからモデルを構築する
        tf.keras.Input(shape=input_shape), # 入力の形状の指定. shape=(時間軸, 12誘導)
        # block1
        tf.keras.layers.Conv1D(64, 7), # 時間方向の1次元畳み込みレイヤー. 32=出力チャネル数, 7=カーネルサイズ
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPool1D(2),
        # block2
        tf.keras.layers.Conv1D(128, 3, strides=2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        # block3
        tf.keras.layers.Conv1D(256, 3, strides=2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        # pooling
        tf.keras.layers.GlobalAveragePooling1D(), # 時間方向のglobal pooling
        # 最終レイヤー
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

model = get_model()
model.summary()

In [ ]:
# モデルのコンパイル (学習条件の設定)
model = get_model(input_shape=(None, 12)) # 時間軸の入力長さNone=可変にして再度model構築
model.compile(optimizer='adam', # オプティマイザーにAdamを指定
              loss='binary_crossentropy', # 損失関数にbinary crossentropyを指定
              metrics=['AUC'], # 評価関数にAUCを指定
             )
# モデルの保存方法の指定
checkpoint_filepath = "weight_fold{}.ckpt".format(fold)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, # 保存path
    save_weights_only=True, # 重みのみを保存
    monitor='val_auc', # validataionのAUCの値に基づいて重みを保存する
    mode='max', # validataionのAUCが最大となった時重みを保存する
    save_best_only=True # AUCが改善したときのみ保存する
)

In [ ]:
# モデルの学習
model.fit(
    train_dataset, # 学習に用いるdataset
    validation_data=valid_dataset, # 検証に用いるdataset
    callbacks=[model_checkpoint_callback], # モデル保存方法の指定
    epochs=16, # epoch数 (1epoch=すべての画像を1回ずつ学習に利用する)
)

In [ ]:
# fold1-4についても学習を行う
# for loopの中身は上記のfold 0での処理と同様です
for fold in range(1,5):
    print("fold: {}".format(fold))
    X_train = ecg_train[folds[fold][0]] # 学習データの入力データを抽出
    y_train = target_train[folds[fold][0]] # 学習データの正解データを抽出
    X_valid = ecg_train[folds[fold][1]] # 検証データの入力データを抽出
    y_valid = target_train[folds[fold][1]] # 検証データの正解データを抽出
    print("len train: {}. len valid: {}".format(len(X_train), len(X_valid)))

    # train dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((
        X_train, # 入力データ
        y_train, # 正解データ
    ))
    train_dataset = train_dataset.shuffle(len(train_dataset), reshuffle_each_iteration=True) # 学習中にデータをシャッフルすることを指定する
    train_dataset = train_dataset.map(augment_fn, num_parallel_calls=AUTOTUNE) # augmentationの適用
    train_dataset = train_dataset.batch(BATCH_SIZE) # データセットをミニバッチ化してモデルに入力することを指定する

    # valid dataset
    valid_dataset = tf.data.Dataset.from_tensor_slices((
        X_valid, # 入力データ
        y_valid, # 正解データ
    ))
    valid_dataset = valid_dataset.batch(BATCH_SIZE) # データセットをミニバッチ化してモデルに入力することを指定する (シャッフルはしない)

    # model構築
    model = get_model(input_shape=(None, 12))
    # モデルのコンパイル (学習条件の設定)
    model.compile(optimizer='adam', # オプティマイザーにAdamを指定
                  loss='binary_crossentropy', # 損失関数にbinary crossentropyを指定
                  metrics=['AUC'], # 評価関数にAUCを指定
                 )
    # モデルの保存方法の指定
    checkpoint_filepath = "weight_fold{}.ckpt".format(fold)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath, # 保存path
        save_weights_only=True, # 重みのみを保存
        monitor='val_auc', # validataionのAUCの値に基づいて重みを保存する
        mode='max', # validataionのAUCが最大となった時重みを保存する
        save_best_only=True # AUCが改善したときのみ保存する
    )
    # モデルの学習
    model.fit(
        train_dataset, # 学習に用いるdataset
        validation_data=valid_dataset, # 検証に用いるdataset
        callbacks=[model_checkpoint_callback], # モデル保存方法の指定
        epochs=16, # epoch数 (1epoch=すべての画像を1回ずつ学習に利用する)
    )

# 学習したモデルを評価する

In [ ]:
# クロスバリデーションのAUCを計算する
preds_valid = np.zeros(len(df_train), np.float32) # 予測結果の代入先
for fold in range(5): # 各foldについて
    print("fold: {}".format(fold))
    # valid dataset
    X_valid = ecg_train[folds[fold][1]] # 検証データの入力データを抽出
    y_valid = target_train[folds[fold][1]] # 検証データの正解データを抽出
    valid_dataset = tf.data.Dataset.from_tensor_slices((
        X_valid, # 入力データ
        y_valid, # 正解データ
    ))
    valid_dataset = valid_dataset.batch(BATCH_SIZE) # データセットをミニバッチ化してモデルに入力することを指定する (シャッフルはしない)

    # 予測
    checkpoint_filepath = "weight_fold{}.ckpt".format(fold)
    model.load_weights(checkpoint_filepath) # 最もvalid AUCが高かったエポックの重みを読み込む
    pred_valid = model.predict(valid_dataset) # 予測の実行
    preds_valid[folds[fold][1]] = pred_valid[:,0] # 予測結果の代入

valid_auc = metrics.roc_auc_score(df_train[col_target], preds_valid)
print("CV: {:.6f}".format(valid_auc))

In [ ]:
# 予測結果のヒストグラムを表示
idx_nega = df_train[col_target].astype(int)==0
idx_posi = df_train[col_target].astype(int)==1
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
plt.hist(preds_valid[idx_nega], bins=np.arange(101)/100, alpha=0.3, label='negative')
plt.hist(preds_valid[idx_posi], bins=np.arange(101)/100, alpha=0.3, label='positive')
plt.legend()
plt.xlabel("predict")
plt.show()

In [ ]:
# labelがpositiveであり予測がpositiveだった心電図を表示
df_tmp = copy.deepcopy(df_train)
df_tmp['pred'] = preds_valid
df_positive = df_tmp[df_tmp[col_target]==1]
df_positive = df_positive.sort_values('pred', ascending=False).reset_index(drop=True)
index = 0 # 表示する行を指定
path_tmp = df_positive['path'][index] # 表示する心電図データのpath
ecg_tmp = np.load(path_tmp) # 心電図データのよみこみ
print("easiest positive case")
print("id: {}".format(df_positive[col_index][index]))
print("predict: {:.6f}".format(df_positive['pred'][index]))
lead_list = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'] # 各誘導の名称
plt.figure(figsize=(20,16))
for i in range(12): # 各誘導を可視化
    if i<6:
        plt.subplot(6,2,i*2+1)
    else:
        plt.subplot(6,2,i*2-10)
    plt.plot(ecg_tmp[:,i])
    plt.xticks(np.arange(0, 801, step=100))
    plt.ylabel(lead_list[i]+"  ", rotation=0, fontsize=16)
    plt.minorticks_on()
    plt.ylim(ecg_tmp.min(),ecg_tmp.max())
    plt.grid(which="major", color="black", alpha=0.5)
    plt.grid(which="minor", color="gray", linestyle=":")

In [ ]:
# labelがpositiveであるのに予測がnegativeだった心電図を表示
df_tmp = copy.deepcopy(df_train)
df_tmp['pred'] = preds_valid
df_positive = df_tmp[df_tmp[col_target]==1]
df_positive = df_positive.sort_values('pred').reset_index(drop=True)
index = 0 # 表示する行を指定
path_tmp = df_positive['path'][index] # 表示する心電図データのpath
ecg_tmp = np.load(path_tmp) # 心電図データのよみこみ
print("most difficult positive case")
print("id: {}".format(df_positive[col_index][index]))
print("predict: {:.6f}".format(df_positive['pred'][index]))
lead_list = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'] # 各誘導の名称
plt.figure(figsize=(20,16))
for i in range(12): # 各誘導を可視化
    if i<6:
        plt.subplot(6,2,i*2+1)
    else:
        plt.subplot(6,2,i*2-10)
    plt.plot(ecg_tmp[:,i])
    plt.xticks(np.arange(0, 801, step=100))
    plt.ylabel(lead_list[i]+"  ", rotation=0, fontsize=16)
    plt.minorticks_on()
    plt.ylim(ecg_tmp.min(),ecg_tmp.max())
    plt.grid(which="major", color="black", alpha=0.5)
    plt.grid(which="minor", color="gray", linestyle=":")

In [ ]:
# labelがnegativeであり予測がnegativeだった心電図を表示
df_tmp = copy.deepcopy(df_train)
df_tmp['pred'] = preds_valid
df_negative = df_tmp[df_tmp[col_target]==0]
df_negative = df_negative.sort_values('pred').reset_index(drop=True)
index = 0 # 表示する行を指定
path_tmp = df_negative['path'][index] # 表示する心電図データのpath
ecg_tmp = np.load(path_tmp) # 心電図データのよみこみ
print("easiest negative case")
print("id: {}".format(df_negative[col_index][index]))
print("predict: {:.6f}".format(df_negative['pred'][index]))
lead_list = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'] # 各誘導の名称
plt.figure(figsize=(20,16))
for i in range(12): # 各誘導を可視化
    if i<6:
        plt.subplot(6,2,i*2+1)
    else:
        plt.subplot(6,2,i*2-10)
    plt.plot(ecg_tmp[:,i])
    plt.xticks(np.arange(0, 801, step=100))
    plt.ylabel(lead_list[i]+"  ", rotation=0, fontsize=16)
    plt.minorticks_on()
    plt.ylim(ecg_tmp.min(),ecg_tmp.max())
    plt.grid(which="major", color="black", alpha=0.5)
    plt.grid(which="minor", color="gray", linestyle=":")

In [ ]:
# labelがnegativeであるのに予測がpositiveだった心電図を表示
df_tmp = copy.deepcopy(df_train)
df_tmp['pred'] = preds_valid
df_negative = df_tmp[df_tmp[col_target]==0]
df_negative = df_negative.sort_values('pred', ascending=False).reset_index(drop=True)
index = 0 # 表示する行を指定
path_tmp = df_negative['path'][index] # 表示する心電図データのpath
ecg_tmp = np.load(path_tmp) # 心電図データのよみこみ
print("most difficult negative case")
print("id: {}".format(df_negative[col_index][index]))
print("predict: {:.6f}".format(df_negative['pred'][index]))
lead_list = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'] # 各誘導の名称
plt.figure(figsize=(20,16))
for i in range(12): # 各誘導を可視化
    if i<6:
        plt.subplot(6,2,i*2+1)
    else:
        plt.subplot(6,2,i*2-10)
    plt.plot(ecg_tmp[:,i])
    plt.xticks(np.arange(0, 801, step=100))
    plt.ylabel(lead_list[i]+"  ", rotation=0, fontsize=16)
    plt.minorticks_on()
    plt.ylim(ecg_tmp.min(),ecg_tmp.max())
    plt.grid(which="major", color="black", alpha=0.5)
    plt.grid(which="minor", color="gray", linestyle=":")

# 提出ファイルの作成

In [ ]:
# test dataに対する予測
preds_test = np.zeros([5, len(df_test)], np.float32) # 予測結果の代入先
for fold in range(5): # 各foldについて
    print("fold: {}".format(fold))
    # valid dataset
    X_valid = ecg_train[folds[fold][1]] # 検証データの入力データを抽出
    y_valid = target_train[folds[fold][1]] # 検証データの正解データを抽出
    valid_dataset = tf.data.Dataset.from_tensor_slices((
        X_valid, # 入力データ
        y_valid, # 正解データ
    ))
    valid_dataset = valid_dataset.batch(BATCH_SIZE) # データセットをミニバッチ化してモデルに入力することを指定する (シャッフルはしない)

    # 予測
    checkpoint_filepath = "weight_fold{}.ckpt".format(fold)
    model.load_weights(checkpoint_filepath) # 最もvalid AUCが高かったエポックの重みを読み込む
    pred_test = model.predict(test_dataset) # 予測の実行
    preds_test[fold] = pred_test[:,0] # 予測結果の代入
print("preds_test.shape: {}".format(preds_test.shape))
print(preds_test)

In [ ]:
### submitファイルを作成
preds_test_mean = preds_test.mean(axis=0) # 各foldのmodelの予測の平均値を最終的な予測結果として採用する
print("preds_test_mean.shape: {}".format(preds_test_mean.shape))
df_sub[col_target] = preds_test.mean(axis=0) # 推定結果を代入
df_sub.to_csv("submission.csv", index=None) # submitファイルを保存
df_sub.head() # 最初の5行を表示

# 次にすること
- model をさらに深くする
- augmentation を加える (noise, amplify, etc)
- ハイパーパラメータ･チューニング (epoch を増やす, learning rate を減らす, etc)
- メタデータを活用する (sex, age, label_type)
- kaggle の解法を参考にする (e.g. https://www.kaggle.com/shayanfazeli/heartbeat)

### 参考リンク
- [心電図データ×機械学習まとめ](https://medium.com/micin-developers/%E5%BF%83%E9%9B%BB%E5%9B%B3%E3%83%87%E3%83%BC%E3%82%BF-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%81%BE%E3%81%A8%E3%82%81-ed702a0008d4)